Ran on Google Colab

```
# This is formatted as code
```



In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets

In [2]:
from torchvision.transforms import ToTensor

In [3]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [4]:
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

In [5]:
batch_size = 64

train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

In [6]:
for X, y in test_dataloader:
  print(f"Shape of X - N x C x H x W: {X.shape}" )
  print(f"Shape of y: {y.shape} {y.dtype}")
  break

Shape of X - N x C x H x W: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [7]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cpu


In [8]:
class NeuralNetwork(nn.Module):
  def __init__(self):
    super(NeuralNetwork, self).__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512, 512),
        nn.ReLU(),
        nn.Linear(512,10)
    )
  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [9]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)


In [10]:
def train(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  model.train()
  for batch, (X, y) in enumerate(dataloader):
    pred = model(X)
    loss = loss_fn(pred, y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch % 100 == 0:
      loss, current = loss.item(), batch * len(X)
      print(f"Loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

In [11]:
def test(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  model.eval()
  test_loss, correct = 0, 0
  with torch.no_grad():
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)
      pred = model(X)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()
  test_loss /= num_batches
  correct /= size
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [12]:
epochs = 20
for t in range(epochs):
  print(f"Epoch {t+1}\n---------")
  train(train_dataloader, model, loss_fn, optimizer)
  test(test_dataloader, model, loss_fn)

print("Done!")

Epoch 1
---------
Loss: 2.320230 [    0/60000]
Loss: 2.296420 [ 6400/60000]
Loss: 2.274446 [12800/60000]
Loss: 2.255741 [19200/60000]
Loss: 2.253388 [25600/60000]
Loss: 2.224165 [32000/60000]
Loss: 2.228979 [38400/60000]
Loss: 2.198999 [44800/60000]
Loss: 2.197954 [51200/60000]
Loss: 2.157129 [57600/60000]
Test Error: 
 Accuracy: 40.4%, Avg loss: 2.154130 

Epoch 2
---------
Loss: 2.173887 [    0/60000]
Loss: 2.158952 [ 6400/60000]
Loss: 2.098130 [12800/60000]
Loss: 2.105741 [19200/60000]
Loss: 2.072710 [25600/60000]
Loss: 2.004449 [32000/60000]
Loss: 2.039327 [38400/60000]
Loss: 1.954915 [44800/60000]
Loss: 1.959691 [51200/60000]
Loss: 1.893024 [57600/60000]
Test Error: 
 Accuracy: 57.0%, Avg loss: 1.886843 

Epoch 3
---------
Loss: 1.921667 [    0/60000]
Loss: 1.890580 [ 6400/60000]
Loss: 1.769601 [12800/60000]
Loss: 1.809258 [19200/60000]
Loss: 1.719548 [25600/60000]
Loss: 1.654234 [32000/60000]
Loss: 1.693523 [38400/60000]
Loss: 1.578229 [44800/60000]
Loss: 1.605666 [51200/60000]
L